# Week_2 Assignment_3: Нормализация признаков

https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/w7Rqc/normalizatsiia-priznakov

Данное задание основано на материалах лекции по линейным методам классификации.

#### Вы научитесь:

- работать с персептроном — простейшим вариантом линейного классификатора
- повышать качество линейной модели путем нормализации признаков

#### Введение

Линейные алгоритмы — распространенный класс моделей, которые отличается своей простотой и скоростью работы. Их можно обучать за разумное время на очень больших объемах данных, и при этом они могут работать с любыми типами признаков — вещественными, категориальными, разреженными. В этом задании мы предлагаем вам воспользоваться [персептроном] — одним из простейших вариантов линейных моделей.

Как и в случае с метрическими методами, качество линейных алгоритмов зависит от некоторых свойств данных. В частности, признаки должны быть нормализованы, то есть иметь одинаковый масштаб. Если это не так, и масштаб одного признака сильно превосходит масштаб других, то качество может резко упасть.

Один из способов нормализации заключается в стандартизации признаков. Для этого берется набор значений признака на всех объектах, вычисляется их среднее значение и стандартное отклонение. После этого из всех значений признака вычитается среднее, и затем полученная разность делится на стандартное отклонение.

#### Реализация в Scikit-Learn

В библиотеке scikit-learn линейные методы реализованы в пакете sklearn.linear_model. Мы будем работать с реализацией персептрона [sklearn.linear_model.Perceptron](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html). Как и у большинства моделей, обучение производится с помощью функции fit, построение прогнозов — с помощью функции predict.

Пример использования:

    import numpy as np
    from sklearn.linear_model import Perceptron
    X = np.array([[1, 2], [3, 4], [5, 6]])
    y = np.array([0, 1, 0])
    clf = Perceptron()
    clf.fit(X, y)
    predictions = clf.predict(X)

В качестве метрики качества мы будем использовать долю верных ответов (accuracy). Для ее подсчета можно воспользоваться функцией sklearn.metrics.accuracy_score, первым аргументом которой является вектор правильных ответов, а вторым — вектор ответов алгоритма.

Для стандартизации признаков удобно воспользоваться классом [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?highlight=sklearn%20preprocessing%20standardscaler#sklearn.preprocessing.StandardScaler). Функция fit_transform данного класса находит параметры нормализации (средние и дисперсии каждого признака) по выборке, и сразу же делает нормализацию выборки с использованием этих параметров. Функция transform делает нормализацию на основе уже найденных параметров.

Пример использования:

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_train = np.array([[100.0, 2.0], [50.0, 4.0], [70.0, 6.0]])
    X_test = np.array([[90.0, 1], [40.0, 3], [60.0, 4]])
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

#### Инструкция по выполнению

1. Загрузите обучающую и тестовую выборки из файлов perceptron-train.csv и perceptron-test.csv. Целевая переменная записана в первом столбце, признаки — во втором и третьем.
2. Обучите персептрон со стандартными параметрами и random_state=241.
3. Подсчитайте качество (долю правильно классифицированных объектов, accuracy) полученного классификатора на тестовой выборке.
4. Нормализуйте обучающую и тестовую выборку с помощью класса StandardScaler. 
5. Обучите персептрон на новой выборке. Найдите долю правильных ответов на тестовой выборке.
6. Найдите разность между качеством на тестовой выборке после нормализации и качеством до нее. Это число и будет ответом на задание.
Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.421. При необходимости округляйте дробную часть до трех знаков.

## Ответы на вопросы задания

In [2]:
# load libraries
import pandas as pd
import numpy as np

### 1. Загрузите обучающую и тестовую выборки из файлов perceptron-train.csv и perceptron-test.csv. Целевая переменная записана в первом столбце, признаки — во втором и третьем

In [5]:
data_train = pd.read_csv('perceptron-train.csv', header=None)
data_train

,0,1,2
0,-1.0,-0.024626,1174.600238
1,1.0,-0.978058,1083.198803
2,-1.0,0.314272,-1472.977609
3,-1.0,0.179752,231.017267
4,1.0,-1.262544,-778.271726
...,...,...,...
295,-1.0,-0.126438,1861.564124
296,-1.0,-0.490972,1815.652064
297,1.0,-0.375750,-314.074374
298,1.0,-1.861976,-1083.677679


In [6]:
data_test = pd.read_csv('perceptron-test.csv', header=None)
data_test

,0,1,2
0,-1.0,1.651437,1337.453826
1,1.0,-0.866495,1191.232457
2,-1.0,0.789828,-475.647768
3,-1.0,0.179549,1959.095353
4,1.0,-0.434351,568.504207
...,...,...,...
195,-1.0,1.150206,2559.850231
196,1.0,-0.732462,159.748695
197,1.0,0.412912,-1587.206427
198,-1.0,-0.098685,682.034777


In [9]:
X_train = data_train.values[:,1:]
y_train = data_train.values[:,0]
X_test = data_test.values[:,1:]
y_test = data_test.values[:,0]

### 2. Обучите персептрон со стандартными параметрами и random_state=241

In [12]:
from sklearn.linear_model import Perceptron
clf = Perceptron(random_state=241)
clf.fit(X_train, y_train)

Perceptron(random_state=241)

### 3. Подсчитайте качество (долю правильно классифицированных объектов, accuracy) полученного классификатора на тестовой выборке

In [18]:
accuracy = clf.score(X_test, y_test)
accuracy

0.655

### 4. Нормализуйте обучающую и тестовую выборку с помощью класса StandardScaler

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 5. Обучите персептрон на новой выборке. Найдите долю правильных ответов на тестовой выборке

In [17]:
clf.fit(X_train_scaled, y_train)
accuracy_scaled = clf.score(X_test_scaled, y_test)
accuracy_scaled

0.725

### 6. Найдите разность между качеством на тестовой выборке после нормализации и качеством до нее. Это число и будет ответом на задание

In [19]:
print("delta = ", accuracy_scaled - accuracy)

delta =  0.06999999999999995


**Ответ: 7%**